In [48]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import csv

csv.field_size_limit(sys.maxsize)
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words("english")

labels = ['fake', 'real']

#Snowballstemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

news_data = pd.read_csv('WELFake_Dataset.csv', engine='python', encoding='utf-8', on_bad_lines='skip').fillna('')

news_data_smaller = news_data.sample(n=10).reset_index(drop=True)

#uncomment for all data
news_data = news_data_smaller


def remove_punctuation(some_string):
    unpuncted_string = some_string.translate(str.maketrans("","",string.punctuation))
    return(unpuncted_string)

def apply_stemming(some_string):
    new_list = []
    for word in some_string.split():
        word = stemmer.stem(word)
        new_list.append(word)
    return(new_list)


news_data["title_clean"] = news_data["title"].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))
news_data["title_clean"] = news_data["title_clean"].apply(remove_punctuation)
news_data["title_clean"] = news_data["title_clean"].apply(apply_stemming)

news_data["text_clean"] = news_data["text"].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))
news_data["text_clean"] = news_data["text_clean"].apply(remove_punctuation)
news_data["text_clean"] = news_data["text_clean"].apply(apply_stemming)

#print(news_data["text_clean"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
print(news_data["title_clean"][0])
print(news_data["text_clean"][0])

['vapen', 'clear', 'meet', 'the', 'world', 'first', 'cannabi', 'inhal']
['post', 'eddi', 'we', 'generat', 'social', 'medic', 'cannabi', 'evolut', 'not', 'number', 'cannabi', 'advoc', 'increas', 'creativ', 'innov', 'around', 'globe', 'develop', 'new', 'idea', 'left', 'right', 'there', 'way', 'ever', 'get', 'medicin', 'social', 'intak', 'marijuana', 'if', 'thought', "you'v", 'seen', 'all', 'though', 'think', 'again', 'the', 'world', 'firstev', 'cannabi', 'inhal', 'start', 'roll', 'goe', 'name', 'vapen', 'clear', 'there', 'noth', 'els', 'like', 'there', 'so', 'what', 'vapen', 'clear', 'well', 'look', 'exact', 'like', 'typic', 'inhal', 'it', 'perform', 'way', 'too', 'without', 'albuterol', 'the', 'vapen', 'clear', 'releas', 'psychoact', 'compound', 'thc', 'it', 'pack', 'power', '10mg', 'expenditur', 'per', 'puff', 'equal', '100', 'total', 'puff', 'per', 'cartridg', 'tone', 'meet', 'need', 'wherea', 'normal', 'vapor', 'pen', 'heat', 'content', 'vapen', 'clear', "doesn't", 'heat', 'thc', 'ra

In [ ]:
def bow_maker(some_list):
    bow = set()
    for word in some_list:
        numbr = some_list.count(word)
        bow.add((word, numbr))
    return(bow)

print(news_data["title_clean"].apply(bow_maker)[0])

{('sanction', 1), ('ahead', 1), ('decis', 1), ('darfur', 1), ('bashir', 1), ('visit', 1), ('us', 1), ('sudan', 1)}


In [50]:
word_frequency = {}
wordPerLine = {}
allwordsAmount = {}
allTokens = []
counter = 0
for line in news_data["text_clean"]:
    #print(line)
    for token in line:
        if token not in wordPerLine.keys():
            wordPerLine[token] = 1
            allwordsAmount[token] = 1
            if token not in allTokens:
              allTokens.append(token)
        else:
            wordPerLine[token] += 1
            allwordsAmount[token] += 1
    word_frequency[counter] = wordPerLine
    wordPerLine = {}
    counter = counter +1

bowResults = []
for line in news_data["text_clean"]:
    vector=[]
    for w in allTokens:
        vector.append(line.count(w))
    print(vector)
    bowResults.append(vector) 

[1, 1, 1, 1, 2, 2, 7, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 5, 1, 1, 7, 1, 1, 1, 1, 8, 7, 1, 1, 3, 2, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 